#### 복합명사화 처리가 끝난 하단의 df파일에서 Noun만 추출 및 기타 전처리(소문자 통일, 한글자 제거)

#### OUTPUT: w4/T1_out1_df_noun_title.csv

In [8]:
import os
import pandas as pd
from tqdm import tqdm

In [3]:
df = pd.read_csv('../../week3/outputs/df0_1_res_totalComps_title.csv', encoding='utf-8-sig')

In [17]:
# 명사 혹은 그에 준하는 token record만 추출

df_title_noun = pd.DataFrame(columns=['art_id','token','type'])

for i in tqdm(range(len(df)), desc='명사(준) 추출 로직 진행중'):
    if df.loc[i,'type']=='NNG' or df.loc[i,'type']=='NNP' or df.loc[i,'type']=='SL'or df.loc[i,'type']=='SH' or df.loc[i,'type']=='comp_Nouns2' or df.loc[i,'type']=='comp_Nouns3' or df.loc[i,'type']=='comp_SN_NNBC_NNG' or df.loc[i,'type']=='comp_SN_NR_NNBC' or df.loc[i,'type']=='comp_SN_NNBC' or df.loc[i,'type']=='comp_SN_MM_NNG':
        df_title_noun = df_title_noun.append(df.loc[i,['art_id','token','type']])

df_noun_title = df_title_noun.reset_index(drop=True)

명사(준) 추출 로직 진행중: 100%|██████████| 111562/111562 [02:13<00:00, 836.73it/s]


In [20]:
df_noun_title.head()

df_noun_title = df_noun_title.astype({'token':'str'})

In [21]:
# SL타입 토큰 중 대문자 포함 token 인식 및 인덱스 저장
temp = 0
SLupStr_index_list = []

def isSL(token) :
    global temp
    global SLupStr_index_list
    if df_noun_title.loc[temp,'type'] =='SL':   
        if (not token.islower()):
            SLupStr_index_list.append(temp) # 인덱스 사용 메서드 name
    temp += 1

df_noun_title.loc[:,'token'].apply(isSL)

SLupStr_index_list[:5]



[5, 60, 64, 69, 92]

In [22]:
#상위 저장한 인덱스 기반, 소문자로 통일 및 저장 : df_noun_title_slLower
df_noun_title_slLower = df_noun_title
df_noun_title_slLower.loc[SLupStr_index_list,'token'] = df_noun_title_slLower.loc[SLupStr_index_list,'token'].apply(lambda x: x.lower())

In [23]:
# 길이가 1인(즉, 한글자 단어) 단어 인덱스 찾기

temp = 0
len1_index_list = []

def isLen1(token) :
    global temp
    global len1_index_list
    if (len(token) <= 1):
        len1_index_list.append(temp) # 인덱스 사용 메서드 name
    temp += 1

df_noun_title_slLower.loc[:,'token'].apply(isLen1)

len1_index_list[:5]


[7, 29, 35, 77, 83]

In [24]:
# 길이가 1인 단어 제거 df 만들기 : df_noun_title_rmLen1

df_noun_title_rmLen1= df_noun_title_slLower.drop(index=len1_index_list)

df_noun_title_rmLen1.head(20)

,art_id,token,type
0,0,erp,SL
1,0,프로그램,NNG
2,0,오류,NNG
3,0,장애조치,comp_Nouns2
4,0,방법,NNG
5,1,pl,SL
6,2,공부,NNG
8,3,개인,NNG
9,3,생각,NNG
10,3,서울,NNP


In [31]:
# 특정 불필요 값들 제거

dropTokens=  ['부탁','개발자','관련','안녕','질문','조언','고민']

df_noun_title_rmLen1 = df_noun_title_rmLen1[~df_noun_title_rmLen1['token'].isin(dropTokens)]

In [32]:
# output1 : 명사 추출 & SL 소문자 통일 & 길이 1짜리 단어 제거 
    # 저장
df_noun_title_rmLen1.to_csv('../outputs/T1_df_cleaned_noun_title.csv',encoding='utf-8-sig', index=False)

df_noun_title_rmLen1.head()

,art_id,token,type
0,0,erp,SL
1,0,프로그램,NNG
2,0,오류,NNG
3,0,장애조치,comp_Nouns2
4,0,방법,NNG
